# Yapay Sinir Ağları (YSA)

In [24]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

from warnings import filterwarnings
filterwarnings('ignore')

In [25]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Çok Katmanlı Algılayıcı

In [26]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

In [27]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [28]:
X_train_scaled = scaler.transform(X_train)

In [29]:
X_test_scaled = scaler.transform(X_test)

In [30]:
mlp_model = MLPRegressor(hidden_layer_sizes = (100,20)).fit(X_train_scaled, y_train)

In [31]:
mlp_model.n_layers_ #Katman Sayisi

4

In [32]:
mlp_model.hidden_layer_sizes #Gizli katman sayisi icindeki eleman sayisi

(100, 20)

# Tahmin

In [33]:
mlp_model.predict(X_train_scaled)[0:5]

array([ 63.72041222, 304.36229527, 208.61746255,  56.90276084,
        53.42365417])

In [34]:
y_pred = mlp_model.predict(X_test_scaled)

In [35]:
np.sqrt(mean_squared_error(y_test, y_pred))

493.5658168294429

# Model Tuning

In [39]:
mlp_params = {'alpha': [0.1, 0.01,0.02,0.005],
             'hidden_layer_sizes': [(20,20),(100,50,150),(300,200,150)],
             'activation': ['relu','logistic']}

In [40]:
mlp_cv_model = GridSearchCV(mlp_model, mlp_params, cv = 10)

In [41]:
mlp_cv_model.fit(X_train_scaled, y_train)

GridSearchCV(cv=10, estimator=MLPRegressor(hidden_layer_sizes=(100, 20)),
             param_grid={'activation': ['relu', 'logistic'],
                         'alpha': [0.1, 0.01, 0.02, 0.005],
                         'hidden_layer_sizes': [(20, 20), (100, 50, 150),
                                                (300, 200, 150)]})

In [42]:
mlp_cv_model.best_params_

{'activation': 'relu', 'alpha': 0.005, 'hidden_layer_sizes': (100, 50, 150)}

In [43]:
mlp_tuned = MLPRegressor(alpha = 0.02, hidden_layer_sizes = (100,50,150))

In [44]:
mlp_tuned.fit(X_train_scaled, y_train)

MLPRegressor(alpha=0.02, hidden_layer_sizes=(100, 50, 150))

In [45]:
y_pred = mlp_tuned.predict(X_test_scaled)

In [46]:
np.sqrt(mean_squared_error(y_test, y_pred))

356.8187742589916